In [16]:
import csv
import pprint
import pandas as pd
import pywikibot
from caphi2ipa import caphipp2ipa

# To do

* Singular nouns:
    * Add plurals
    * Handle "complex" nouns:
        * Several glosses (';')
        * Complex glosses ('_' or ' ')
    * Add IPA
* Plural nouns
* Adjectives
* Active participles (NOUN_ACT:MS)
* Passive participles (NOUN_PASS)
* INTERJ
* ADV
* NOUN:PHRASE
* Verbs
* VERB:PHRASE
* Issues:
    * شرعية: WARNING: API warning (edit): The value passed for "text" contains invalid or non-normalized data. Textual data should be valid, NFC-normalized Unicode without C0 control characters other than HT (\t), LF (\n), and CR (\r).

# Read TSV

In [18]:
maknuune_file = "maknuune-v1.0.1.tsv"

singular_nouns = ['NOUN:MS', 'NOUN:FS']

df = pd.read_csv(maknuune_file,sep='\t')

# Stats by PoS

In [19]:
df_count = df.groupby(['ANALYSIS'])['ANALYSIS'].count()

In [20]:
df_count.loc[lambda x : x > 50].sort_values(ascending=False)

ANALYSIS
VERB:I             6345
VERB:C             6329
VERB:P             6191
NOUN:MS            4183
NOUN:FS            3050
ADJ:MS             2388
NOUN:PHRASE        2343
NOUN:P             1894
NOUN_ACT:MS         759
VERB:PHRASE         639
NOUN_PASS           305
ADJ:P               288
ADJ:PHRASE          249
NOUN_ACT:PHRASE     117
ADJ/NOUN            108
INTERJ              106
ADJ:FS               78
ADJ_COMP             77
ADV                  71
NOUN_PROP            65
Name: ANALYSIS, dtype: int64

# First: singular nouns deriving from a root and without notes 

In [21]:
# 1355 entries with notes => deal with them later
# df.loc[df['NOTES'].notnull()].count()

# 897 entries with NTWS roots => deal with them later
# df.loc[df['ROOT'] == 'NTWS'].count()

# df['LEMMA_BW'] == df['FORM_BW'] to exclude weird cases for nouns

df_singular_nouns = df.loc[(df['ROOT'] != 'NTWS') & (df['ANALYSIS'].isin(singular_nouns)) & (df['LEMMA_BW'] == df['FORM_BW']) & (df['NOTES'].isnull())]

In [7]:
df_singular_nouns

,ID,ROOT,ROOT_NTWS,ROOT_1,LEMMA,LEMMA_SEARCH,FORM,LEMMA_BW,FORM_BW,CAPHI++,ANALYSIS,GLOSS,GLOSS_MSA,EXAMPLE_USAGE,NOTES,SOURCE,ANNOTATOR,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1,ء.ب.د,NaN,ء,أَبَد,أبد,أَبَد,>abad,>abad,2 a b a d,NOUN:MS,eternity;forever,NaN,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN
1,2,ء.ب.ر,NaN,ء,إِبْرِة,إبرة,إِبْرِة,<iborip,<iborip,2 i b r e,NOUN:FS,needle;injection,إِبْرَة#حُقْنَة,في إِبْرِة وقعت تحت الكنب دير بالك.#أخذت ابرة ...,NaN,NaN,شهد دعباس,NaN,NaN,NaN
7,8,ء.ب.ط,NaN,ء,أَبَاط,أباط,أَبَاط,>abaAT,>abaAT,2 a b aa t.,NOUN:MS,armpit,إِبْط,ولك ليش أباطك أسود هيك؟,NaN,NaN,شهد دعباس,NaN,NaN,NaN
8,9,ء.ب.ط,NaN,ء,بَاط,باط,بَاط,baAT,baAT,b aa t.,NOUN:MS,armpit,إِبْط,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN
10,11,ء.ب.و,NaN,ء,أَب,أب,أَب,>ab,>ab,2 a b,NOUN:MS,father,أب,الله يرحمه أبوه بقى زلمة مليح وبينشد فيه الظهر,NaN,NaN,شهد دعباس,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36039,36263,ي.م.ن,NaN,ي,يَمِين,يمين,يَمِين,yamiyn,yamiyn,y a m ii n,NOUN:MS,right,يمين,روتش لك شوي لليمين,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36040,36264,ي.م.ن,NaN,ي,يَمِين,يمين,يَمِين,yamiyn,yamiyn,y a m ii n,NOUN:MS,vow;oath,NaN,احلف يَمِين إِنه مالكاش دخَّل!,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36044,36268,ي.م.ن,NaN,ي,يُمُن,يمن,يُمُن,yumun,yumun,y u m u n,NOUN:MS,blessing,نِعْمَة,ألف مبارك الشهر وكل عام وانتو بخير وان شاء الل...,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36056,36280,ي.و.م,NaN,ي,يَومِيِّة,يومية,يَومِيِّة,yawmiy~ip,yawmiy~ip,y oo m i y y e,NOUN:FS,daily_payment,دفع بنظام المياومة,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN


# Wiktionary format

In [22]:
beginning_file = """'''User:A455bcd9/Maknuune/{entry}'''
{{{{-start-}}}}"""

template = """==South Levantine Arabic==
{{{{bulk import|{{{{R:ajp:Maknuune|id={id}}}}}}}}}
{{{{ajp-root|{root}}}}}

===Pronunciation===

* {{{{ajp-IPA|{ipa}}}}}

===Noun===
{{{{ajp-noun|head={head}|g={gender}|tr={transliteration}}}}}

# {glosses}{example}

===References===
* {{{{R:ajp:Maknuune}}}}"""

end_file="{{{{-stop-}}}}"

In [23]:
site = pywikibot.Site()

# Select 10 random entries

In [ ]:
# df_singular_nouns = df_singular_nouns.sample(n=1000)

# CAPHI++ to Wikt

In [24]:
for index, row in df_singular_nouns.iterrows():
    caphipp = row['CAPHI++'].strip()
    ipa = caphipp2ipa(caphipp, mode='wikt')
    # print(row['LEMMA'], '=>', caphipp, '=>', ipa)
    
# test m a w J uu d => mawjūd (موجود)
# maw(dʒ)uːd // مَوْ ُجود
print(caphipp2ipa('m a w J uu d', mode='wikt'))
# 2 i sh t aa Q => ištāʔ (اشتاق)
# ʔiʃtaː(q)
print(caphipp2ipa('2 i sh t aa Q', mode='wikt'))

mawjūd
ʔištāʔ


# Test entries

In [25]:
test_entries = [
    'أب',
    'أباط',
    'أجار',
    'أجرة',
    'أخذ',
    'ألفة',
    'إيجار',
    'باط',
    'بلع',
    'تأثير',
    'تأجير',
    'تأجيل',
    'تجارة',
    'خرقة',
    'زهاب',
    'عنصرية',
    'غواصة',
    'قذلة',
    'كماجة',
    'لقمة',
    'مهاجر',
    'مواخذة',
    'شرعية',
    'بيئة',
    'كيلو',
    'مفاد',
    'خضرة',
    'نظرة',
    'فناء',
    'إفشاء',
    'عجلة',
    'صحصحة',
    'واعظ',
]

# Save entries in Wiktionary

## Clean glosses

In [ ]:
def clean_gloss(gloss):
    clean_gloss = gloss
    # a_type_of_jar_for_keeping_water_cold => 'jar for keeping water cold'
    # see_phrase => ? don't do anything?
    # sb => somebody
    # some one ?
    # () => what to do with them?
    # ceramic work.
    # sth similar to a container that is made by sewing several "7 u s. u r" together. The farmers usually keep wheat in it.
    # sb who eats food off the ground
    # a small ball
    # the state of holding something over sb's head
    # a fat person
    # a single person
    # a narrow path
    # the grated food
    # the beautiful woman
    # a difficult  situation (double space)
    # sb/sth
    # sb's
    # the_person_whose_job_is_to_tell_stories_to_the_people
    # a_stone_inside_a_hole => 'stone inside a hole'
    # It is the ritual animal sacrifice of a livestock animal in the first three days of the funeral that is believed to give solace to the deceased person
    # It is a traditional type of dessert that is made of dough, water and molasses. It is usually eaten with dried figs.
    # a vessel that is usually of brass for kohl.
    # A thin layer of freezing water in the morning, due to lower temperatures.
    # an individual.
    
    return clean_gloss.strip()

In [29]:
i = 0
for index, row in df_singular_nouns.iterrows():
    # remove _[auto] from entries generated automatically from other entries
    clean_gloss_row = row['GLOSS'].replace('_[auto]', '').strip()
    # filter out more complex glosses
    if '_' not in clean_gloss_row and ' ' not in clean_gloss_row:
        entry_values = {}
        entry_values['id'] = row['ID']
        entry_values['entry'] = row['LEMMA_SEARCH'].strip()
        entry_values['root'] = row['ROOT'].strip().replace('.', ' ')
        entry_values['head'] = row['FORM'].strip()
        entry_values['gender'] = 'f' if row['ANALYSIS'] == 'NOUN:FS' else 'm'

        caphipp = row['CAPHI++'].strip()
        entry_values['transliteration'] = caphipp2ipa(caphipp, mode='wikt')
        entry_values['ipa'] = entry_values['transliteration']
        
        entry_values['example'] = ''
        if not pd.isna(row['EXAMPLE_USAGE']):
            example = row['EXAMPLE_USAGE'].strip()
            # Fill example if not null
            if example != '':
                entry_values['example'] = '\n#: {{{{ux|ajp|{example}}}}}'.format(example=example)
        
        # Separate glosses
        glosses = set(clean_gloss_row.split(';'))
        
        entry_values['glosses'] = '[[' + ']], [['.join(glosses) + ']]'

        new_entry = template.format(**entry_values)

        page_name = u"User:A455bcd9/Maknuune/{entry}".format(entry=entry_values['entry'])
        page = pywikibot.Page(site, page_name)

        if len(glosses) > 1 and entry_values['example'] != '':
            page.text = new_entry
            # print(new_entry)
            # page.save(u"Entries with multiple glosses and examples")
            # i += 1
    
    else:
        for g in set(clean_gloss_row.split(';')):
            #if len(g.split('_')) > 4:
            if '.' in g:
                print(g.replace('_', ' ').replace(' sth', ' something'))
                # https://en.wiktionary.org/w/api.php?action=query&titles=test&format=json
        # rummaging_through
        # It_is_what_is_placed_on_the_back_of_the_walking_animal_for_the_purpose_of_riding_on_it

            
    if i > 10:
        break

print('end')

It is an  alternative form of the cloak made of white linen fabric or pure cotton.
0.5 pound
Imam is an Islamic leadership position. Imam is usually the person who leads the prayer
It is like a basket made of rubber, or of wicker and fiber, and has two loops that are used as handles for carrying it. People use it to carry multiple things.
It is what is placed on the back of the walking animal for the purpose of riding on it. It is somewhat similar to a horse's saddle
sweetmeat of baked semolina, soaked in syrup.
It is a traditional type of dessert that is made of dough, water and molasses. It is usually eaten with dried figs.
a cloak, but shorter and with many types.
a handkerchief with a frame surrounded by flowers of different shapes, and above the handkerchief a scarf is placed on the head.
It is a traditional dish that is made of crushed lentils, fried onions, olive oil and dried Mulukhiyah.
It is a traditional dish that is made of crushed lentils, fried onions, olive oil and dried

it is the only egg that is left in the hen house. When the hens lay their eggs, all of the eggs are taken with the exception of one that is left to mark the territory for the other hens to lay their eggs in that place, so that the eggs are not scattered everywhere.
A deep bowl, narrow from the bottom with a wide door, and it is used for eating. It is made of black pottery and comes in several sizes.
2 i t. f aa 7"
2 i t. f aa 7"
It is like a jar that is used to keep oil, milk, yogurt, etc. Its height is almost 60-80 cm and it width is almost 40 cm.
it is a piece of wood that has a nail attache to its end. It is used to prickle the ox in ploughing.
it is a bag where people (mainly soldiers or passengers) keep their provisions in it.
the man whose job is to wake the people up to have their "s u 7 uu r" in Ramadan (i.e. the last meal taken before daybreak during Ramadan)
a magical item that has some deviated Quraanic inscriptions. They aim at preventing marriage, causing illness, bringing

an individual.
A dish consisting of wheat grains and fatty meat and could be served as a soup.
It is a dish that is made of fried potatoes with eggs, salt and some black pepper.
ring shaped pieces of bread or biscuits that are made from many ingredients. The flour, sugar, olive oil, (ghee optional), milk are mixed together. Anise, sesame and black cumin are then added to the mixture. The dough is left to rest for one hour. After that, it is made into dough balls that are placed and flattened into a large baking tray (with a lot of oil). It is saturated with oilve oil.
a person who works in a place like a "bakery" in which there is a large oven where all the people living in the neighbourhood to bake their food in it.
release (negative feelings e.g. anger or sadness)
it is a Palestinian tradition where people visit the bride's house and bring her some utensils that she will use in her kitchen. This visit is usually made after one week of the wedding ceremony.
Islamic jurisprudence.
slam

it is a small place like a pergola that is built from stones and covered with leaves. Usually, the guards stay in it in order to watch the place very carefully.
it is a small place like a pergola that is built from stones and covered with leaves. Usually, the guards stay in it in order to watch the place very carefully.
Niqab  is a garment that covers the face. It is worn by many Muslim women
flatbread topped with a variety of possibilities such as za'atar (thyme), cheese, etc.
A dessert consisting of: semolina, maragrine, sugar, butter, yoghurt, flour, baking powder, blossom water and sugar syrup.
blast of hot air or fire etc.
singing some traditional songs for the kids (the mother usually does it to threaten the kid to sleep early. That's why she sings in a very loud and audible voice)
A conical shaped brass tool, with a thick, strong base and a flat, thick stick used for grinding beans.
A conical shaped brass tool, with a thick, strong base and a flat, thick stick used for grinding 